### A notebook evaluating a possible implementation for _live Boheh plots in a notebook cell_

In [1]:
# some python modules for live data simulation (DataSource)
import math
import random
import numpy as np
import logging
# bokeh plots data streaming API (see plots.py)
from plots import ChannelData, DataSource, BoxSelectionManager
from plots import Scale, ImageChannel
from plots import DataStream, DataStreamer, DataStreamerController

In [2]:
# 2D (i.e. image data source)
class XYDS(DataSource):
    
    def __init__(self, name, iw=100, ih=500):
        DataSource.__init__(self, name)
        self._selection = None
        self._inc = 1
        self._current_index = 0
        self._iw, self._ih = iw, ih
        x, y = np.linspace(0, 10, self._iw), np.linspace(0, 10, self._ih)
        xx, yy = np.meshgrid(x, y)
        self._full_image = np.sin(xx) * np.cos(yy)

    @property
    def width(self):
        return self._iw
    
    @property
    def height(self):
        return self._ih
    
    def pull_data(self):
        cd = ChannelData(self.name)    
        i = self._current_index
        cd.buffer = self._full_image[0:i+1, :]
        self._current_index += self._inc
        if self._current_index > self._ih:
            self._current_index = self._ih
        return cd
    
    def scb(self, selection):
        self._selection = selection
        
    def rcb(self):
        self._selection = None

In [3]:
# Model (i.e plot) properties for the image channel
def img_model_props(this):
    xshsp = dict()
    xshsp['image_shape'] = (this.height, this.width)
    xshsp['start'] = - this.width / 2
    xshsp['end'] = + this.width / 2
    xshsp['num_points'] = this.width
    xshsp['label'] = 'x-axis'
    xshsp['unit'] = 'mm'
    yshsp = dict()
    yshsp['start'] = - this.height / 2
    yshsp['end'] = + this.height / 2
    yshsp['num_points'] = this.height
    yshsp['label'] = 'y-axis'
    yshsp['unit'] = 'mm'
    imp = dict()    
    imp['width'] = 900
    imp['height'] = 250
    imp['x_scale'] = Scale(**xshsp)
    imp['y_scale'] = Scale(**yshsp)
    imp['selection_manager'] = BoxSelectionManager(selection_callback=this.scb, reset_callback=this.rcb)
    return imp

In [4]:
def open_plots():
    # ImageChannel (supports only one DataSource)
    ids = XYDS("is0")
    ic = ImageChannel("ic", data_source=ids, model_properties=img_model_props(this=ids))
    # DataStream (has with multiple Channel support)
    s1 = DataStream('s1', channels=[ic])
    # DataStreamer (has with multiple DataStream support)
    m1 = DataStreamer('m1', data_streams=[s1], update_period=0.50)
    # DataStreamerController (optional widgets to control the DataStreamer)
    DataStreamerController('c1', m1)
    # setup logging
    import logging
    l = logging.getLogger('fs.client.jupyter')
    l.setLevel(logging.DEBUG)

In [5]:
open_plots()

[2017-06-27 15:30:21,465] fs.client.jupyter: DataStreamer.__stop_bokeh_server.m1 <<in
[2017-06-27 15:30:21,466] fs.client.jupyter: DataStreamer.__uninstall_periodic_callbacks.m1 <<in
[2017-06-27 15:30:21,467] fs.client.jupyter: DataStreamer.__uninstall_periodic_callbacks.m1 out>> [took: 1.06 ms]
[2017-06-27 15:30:21,468] fs.client.jupyter: DataStreamer.__clear_models.m1 <<in
[2017-06-27 15:30:21,470] fs.client.jupyter: DataStreamer.__clear_models.m1 out>> [took: 2.74 ms]
[2017-06-27 15:30:21,472] fs.client.jupyter: stopping Bokeh server...
[2017-06-27 15:30:21,475] fs.client.jupyter: Bokeh server stopped & cleanup done
[2017-06-27 15:30:21,477] fs.client.jupyter: DataStreamer.__stop_bokeh_server.m1 out>> [took: 12.33 ms]


In [6]:
import logging
l = logging.getLogger('fs.client.jupyter')
l.setLevel(logging.DEBUG)

[2017-06-27 15:30:15,289] fs.client.jupyter: DataStream.update.s1 <<in
[2017-06-27 15:30:15,296] fs.client.jupyter: DataStream.update.s1 out>> [took: 7.11 ms]
[2017-06-27 15:30:15,790] fs.client.jupyter: DataStream.update.s1 <<in
[2017-06-27 15:30:15,797] fs.client.jupyter: DataStream.update.s1 out>> [took: 6.84 ms]
[2017-06-27 15:30:16,293] fs.client.jupyter: DataStream.update.s1 <<in
[2017-06-27 15:30:16,301] fs.client.jupyter: DataStream.update.s1 out>> [took: 7.17 ms]
[2017-06-27 15:30:16,798] fs.client.jupyter: DataStream.update.s1 <<in
[2017-06-27 15:30:16,805] fs.client.jupyter: DataStream.update.s1 out>> [took: 7.11 ms]
[2017-06-27 15:30:17,300] fs.client.jupyter: DataStream.update.s1 <<in
[2017-06-27 15:30:17,307] fs.client.jupyter: DataStream.update.s1 out>> [took: 6.92 ms]
